In [2]:
import json

# Read JSON file
with open('../data.json', 'r') as json_file:
    matches = json.load(json_file)

In [3]:
matches[0].keys()

dict_keys(['match_day_number', 'match_day_date', 'team_home_name', 'team_home_logo_url', 'team_home_goals', 'team_away_goals', 'team_away_name', 'team_away_logo_url', 'local_points', 'away_points', 'played', 'result', 'field_name', 'referees', 'staff', 'players'])

In [4]:
from rich import print

team_name = "A.D. FERROVIARIA"
# team_name = "C.F. MADRID RIO 'B'"
# team_name = "PASILLO VERDE ARGANZUELA"
# team_name = "C.D. ASOC RETIRO SUR"
players_stats = {}
matches_played = 0

for match in matches:
    if match['team_home_name'] == team_name or match['team_away_name'] == team_name:
        role = 'local' if match['team_home_name'] == team_name else 'away'
        # print(f"Jornada {match['match_day_number']} ({match['match_day_date']})")
        # print(f"{match['team_home_name']} VS {match['team_away_name']}")
        if match['players']:
            for i, player in enumerate(match['players'][f'starting_{role}_players'] + match['players'][f'substitute_{role}_players']):
                if player['name'] not in players_stats:
                    players_stats[player['name']] = {
                        'goals': 0,
                        'own_goals': 0,
                        'yellow_cards': 0,
                        'red_cards': 0,
                        'minutes_played': 0,
                        "penalty_goals": 0,
                        'matches_played': 0,
                        'starting': 0,
                        'substitute': 0
                    }
                players_stats[player['name']]['goals'] += len(player['goals'])
                players_stats[player['name']]['yellow_cards'] += len(player['yellow_cards'])
                players_stats[player['name']]['red_cards'] += len(player['red_cards'])
                players_stats[player['name']]['minutes_played'] += player['minutes_played']
                players_stats[player['name']]['penalty_goals'] += len(player['penalty_goals'])
                players_stats[player['name']]['own_goals'] += len(player['own_goals'])
                players_stats[player['name']]['matches_played'] += 1
                if i < 11:
                    players_stats[player['name']]['starting'] += 1
                else:
                    players_stats[player['name']]['substitute'] += 1
        if match['played']:
            matches_played += 1
                
# print(players_stats)

In [5]:
import pandas as pd
from pandasql import sqldf

players_stats_list = []

for player_name in players_stats.keys():
    players_stats_list.append({
        'name': player_name,
        'goals': players_stats[player_name]['goals'],
        'own_goals': players_stats[player_name]['own_goals'],
        'yellow_cards': players_stats[player_name]['yellow_cards'],
        'red_cards': players_stats[player_name]['red_cards'],
        'minutes_played': players_stats[player_name]['minutes_played'],
        'penalty_goals': players_stats[player_name]['penalty_goals'],
        'matches_played': players_stats[player_name]['matches_played'],
        'starting': players_stats[player_name]['starting'],
        'substitute': players_stats[player_name]['substitute']
    })

df = pd.DataFrame(players_stats_list)
pysqldf = lambda q: sqldf(q, globals())

### Team players stats

In [6]:
players_stats_table = pysqldf(f"""
SELECT 
    *,
    minutes_played/matches_played as avg_minutes_played_per_match,
    minutes_played/{matches_played} as avg_minutes_played_in_total
FROM 
    df 
ORDER BY 
    minutes_played DESC
""")
players_stats_table

,name,goals,own_goals,yellow_cards,red_cards,minutes_played,penalty_goals,matches_played,starting,substitute,avg_minutes_played_per_match,avg_minutes_played_in_total
0,"LOPEZ DIEZ, ANGEL",1,0,5,0,2680,0,31,30,1,86,78
1,"BRUNS, FLAVIO",3,0,0,0,2495,0,28,28,0,89,73
2,"TELLEZ RUBIO, FRANCISCO JAVIER",0,0,3,0,2467,0,31,29,2,79,72
3,"MORAN ROBLEDILLO, CARLOS",5,0,3,1,2192,0,26,26,0,84,64
4,"ORTEGA GONZALEZ, DAVID EUGENIO",1,0,7,0,2081,0,27,23,4,77,61
5,"SUAREZ SANCHEZ, ADRIAN",0,0,2,0,2070,0,32,23,9,64,60
6,"LOPEZ PEREZ, GONZALO",10,0,5,0,1773,1,24,21,3,73,52
7,"MARCHAN ALVAREZ, ALEJANDRO",6,0,1,0,1762,0,30,19,11,58,51
8,"GOMEZ HERNANDEZ, ADRIAN",0,0,5,0,1719,0,22,20,2,78,50
9,"FERRERO LOPEZ, JESUS RUBEN",9,0,1,0,1649,0,26,22,4,63,48


### Scoring ratio

In [13]:
pysqldf("""
SELECT 
    name, 
    goals,
    penalty_goals,
    goals + penalty_goals as total_goals,
    minutes_played,
    CAST(goals + penalty_goals AS float)/CAST(minutes_played AS float) as avg_goals_per_minute
FROM 
    players_stats_table
WHERE
    goals > 0
ORDER BY
    avg_goals_per_minute DESC
""")

,name,goals,penalty_goals,total_goals,minutes_played,avg_goals_per_minute
0,"GIMENEZ BUENO, MARIO ADRIAN",10,1,11,1232,0.008929
1,"SANCHEZ LUIS, ALVARO",9,0,9,1360,0.006618
2,"LOPEZ PEREZ, GONZALO",10,1,11,1773,0.006204
3,"FERRERO LOPEZ, JESUS RUBEN",9,0,9,1649,0.005458
4,"PEREZ ORTEGA, ALEJANDRO",7,0,7,1495,0.004682
5,"RODRIGUEZ MARTIN, JORGE",3,0,3,661,0.004539
6,"SANCHEZ BONILLA, VICTOR",4,0,4,1016,0.003937
7,"GALAN CARREIRA, MARCOS",1,0,1,272,0.003676
8,"ARROYO RODRIGUEZ, DAVID",5,1,6,1638,0.003663
9,"MARCHAN ALVAREZ, ALEJANDRO",6,0,6,1762,0.003405
